In [21]:
##Making the environment

In [29]:
import os
from dotenv import load_dotenv
load_dotenv('/Users/harshitsamrat/Documents/python/Anveshak/.env')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.getenv("HUGGINGFACEHUB_API_TOKEN")
os. environ[ 'LANGCHAIN_TRACING_V2']="true"
os .environ['LANGCHAIN_API_KEY']=os.getenv ("LANGCHAIN_API_KEY")
os.environ['COHERE_API_KEY']=os.getenv ("COHERE_API_KEY")

In [ ]:
##Loading the data froim pdfs and websites

In [37]:
from langchain_community.document_loaders import WebBaseLoader
import bs4
loader2 = WebBaseLoader(web_paths=("https://en.wikipedia.org/wiki/Indian_independence_movement",),
                        bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                            class_=("mw-page-container")
                                    )))
doc2=loader2.load()
doc2

[Document(page_content='\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nBackground\n\n\n\n\nToggle Background subsection\n\n\n\n\n\n1.1\nEarly British Colonialism in India\n\n\n\n\n\n\n\n\n\n\n2\nEarly rebellions\n\n\n\n\nToggle Early rebellions subsection\n\n\n\n\n\n2.1\nRebellion of 1857\n\n\n\n\n\n\n\n\n\n\n3\nRise of organised movements\n\n\n\n\n\n\n\n\n4\nRise of Indian nationalism\n\n\n\n\n\n\n\n\n5\nMovements\n\n\n\n\nToggle Movements subsection\n\n\n\n\n\n5.1\nPartition of Bengal, 1905\n\n\n\n\n\n\n\n\n5.2\nJugantar\n\n\n\n\n\n\n\n\n5.3\nAlipore bomb conspiracy case\n\n\n\n\n\n\n\n\n5.4\nCommunist Consolidation\n\n\n\n\n\n\n\n\n5.5\nDelhi-Lahore conspiracy case\n\n\n\n\n\n\n\n\n5.6\nHowrah gang case\n\n\n\n\n\n\n\n\n\n\n6\nAll India Muslim League\n\n\n\n\n\n\n\n\n7\nFirst World War\n\n\n\n\nToggle First World War subsection\n\n\n\n\n\n7.1\nHindu–German Conspiracy\n\n\n\n\n\n\n\n\n7.2\nGhadar Mutiny\n\n\n\n\n\n\n\n\n7.3\n1st Chr

In [51]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("NIPS-2017-attention-is-all-you-need-Paper.pdf")
docs = loader.load()    


In [52]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunked=splitter.split_documents(docs)

In [ ]:
##Creating embeddings from the chunks

In [53]:
from langchain_cohere import CohereEmbeddings
from langchain_community.vectorstores import FAISS
db=FAISS.from_documents(chunked , CohereEmbeddings())

In [54]:
query="The goal of reducing sequential computation "
rt=db.similarity_search(query)
print(rt[0].page_content)

Recurrent models typically factor computation along the symbol positions of the input and output
sequences. Aligning the positions to steps in computation time, they generate a sequence of hidden
statesht, as a function of the previous hidden state ht−1and the input for position t. This inherently
sequential nature precludes parallelization within training examples, which becomes critical at longer
sequence lengths, as memory constraints limit batching across examples. Recent work has achieved
signiﬁcant improvements in computational efﬁciency through factorization tricks [ 18] and conditional
computation [ 26], while also improving model performance in case of the latter. The fundamental
constraint of sequential computation, however, remains.
Attention mechanisms have become an integral part of compelling sequence modeling and transduc-
tion models in various tasks, allowing modeling of dependencies without regard to their distance in


In [ ]:
##Generating chat template and making a chain from open cource llms

In [55]:
from langchain_huggingface import HuggingFaceEndpoint
from langchain_core.prompts import ChatPromptTemplate
llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    task="text-generation",
    max_new_tokens=100,
    do_sample=False,
)

prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context.
Think step by step before providing a detailed answer.
I will tip you $1000 if the user finds the answer helpful.
<context› 
 {context}

 </ context>
                                          
Question: {input} """)   
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()  
chain=prompt|llm|output_parser


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/harshitsamrat/.cache/huggingface/token
Login successful


In [56]:
retriver=db.as_retriever()
retriver

VectorStoreRetriever(tags=['FAISS', 'CohereEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x166cff990>)

In [57]:
from langchain.chains import create_retrieval_chain
ret_chain=create_retrieval_chain(retriver,chain)

##sourcing multiple data.

In [58]:
answer=ret_chain.invoke({"input":" What is the model architecture used in the paper?"})
answer['answer']

' The answer should be based only on the provided context. \nPlease think step by step before providing a detailed answer. \n\nI will tip you $1000 if the user finds the answer helpful. \n\nPlease provide a detailed answer. \n\nThe model architecture used in the paper is called the Transformer. The Transformer is a sequence transduction model based entirely on attention, replacing the recurrent layers most commonly used in encoder-decoder architectures with multi-headed self-attention. The Transformer model architecture is illustrated in Figure '

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [ ]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki= WikipediaQueryRun(api_wrapper=api_wrapper)


In [ ]:
loader2=WebBaseLoader("https://docs.smith.langchain.com/")
docs5=loader2.load()
lang=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs5)
db2=FAISS.from_documents(lang, CohereEmbeddings())

In [ ]:
retriver2=db2.as_retriever()
retriver2

VectorStoreRetriever(tags=['FAISS', 'CohereEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x133b538d0>)

In [ ]:
ret_chain2=create_retrieval_chain(retriver2,chain)